In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

In [26]:
directory = os.path.abspath("") + '/dataset/ethereum/'
data = pd.read_csv(directory + 'data.csv', on_bad_lines = 'skip')
ECDSA = data.copy()

In [27]:
# Drop blocks with no transactions
ECDSA = ECDSA[ECDSA['TotalTx'] != 0]

# Average block size in bytes - block header size (512 bytes)
ECDSA['BlockSize'] = ECDSA['BlockSize'] - 512

ECDSA['AvgTxSize'] = ECDSA['BlockSize'] / ECDSA['TotalTx']

# Max No of Transactions possible in a 12 MB
ECDSA['NoMaxTxPossible'] = (12 * 1000000) / ECDSA['AvgTxSize']

ECDSA.describe()

,BlockNumber,Timestamp,BlockReward,BlockSize,TotalTx,GasUsed,GasAvgPrice,BlockTime,BaseFeePerGas,AvgTxSize,NoMaxTxPossible
count,2.693665e+06,2.693665e+06,2.693665e+06,2.693665e+06,2.693665e+06,2.693665e+06,2.693665e+06,2.693665e+06,2.693665e+06,2.693665e+06,2.693665e+06
mean,1.780538e+07,1.690694e+09,8.210744e-02,1.396076e+05,1.488773e+02,1.512352e+07,3.341400e-08,1.212839e+01,2.797172e-08,9.842925e+02,1.931178e+04
std,7.796217e+05,9.457689e+06,3.078260e-01,1.227419e+05,5.691164e+01,5.336105e+06,3.027426e-08,1.255729e+00,2.224686e-08,9.271744e+02,1.125013e+04
min,1.645497e+07,1.674302e+09,0.000000e+00,1.210000e+02,1.000000e+00,2.100000e+04,4.724384e-09,1.200000e+01,3.996741e-09,1.110508e+02,1.849682e+02
25%,1.713023e+07,1.682510e+09,2.459373e-02,6.163600e+04,1.200000e+02,1.162459e+07,1.803205e-08,1.200000e+01,1.493489e-08,4.299417e+02,9.636653e+03
50%,1.780544e+07,1.690715e+09,4.236888e-02,1.021630e+05,1.400000e+02,1.399854e+07,2.628430e-08,1.200000e+01,2.220459e-08,6.823379e+02,1.758659e+04
75%,1.848048e+07,1.698881e+09,7.913076e-02,1.768990e+05,1.640000e+02,1.731431e+07,3.973690e-08,1.200000e+01,3.396361e-08,1.245246e+03,2.791076e+04
max,1.915573e+07,1.707059e+09,1.822103e+02,2.332655e+06,1.366000e+03,3.002708e+07,9.472830e-06,9.600000e+01,8.132266e-07,6.487600e+04,1.080586e+05
